<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 2<br/>*POS taggers* pour le français dans spaCy et NLTK

**Objectif**

Comparer l'étiqueteur morphosyntaxique français prêt-à-l'emploi de spaCy avec deux étiqueteurs entraînés, l'un dans spaCy et l'autre dans NLTK.

## 1. Installation et test de spaCy

La boîte à outils spaCy est une librairie Python *open source* pour le TAL, dédiée à un usage en production. Les documents suivants vous seront utiles :
* comment [installer](https://spacy.io/usage) spaCy
* comment [télécharger un modèle](https://spacy.io/usage/models) pour une langue donnée (on appelle ces modèles des *trained pipelines* car ils enchaînent plusieurs traitements)
* comment faire les [premiers pas](https://spacy.io/usage/spacy-101) dans l'utilisation de spaCy

Veuillez installer spaCy, puis la *pipeline* pour le français appelée `fr_core_news_sm`.  Si vous utilisez *conda*, installez spaCy dans l'environnement du cours TAL.

In [2]:
import spacy
import tqdm # permet l'affichage d'une barre de progression

**1a.** Une pipeline effectue un ensemble de traitements d'un texte en lui ajoutant des annotations.  Les traitements effectués par la pipeline `fr_core_news_sm` sont [documentés ici](https://spacy.io/models/fr#fr_core_news_sm).  La liste des traitements d'une pipeline figure dans son attribut `.pipe_names`.  On peut activer ou désactiver un traitement T avec, respectivement, les méthodes `.disable_pipe(T)` et `.enable_pipe(T)` appliquées à la pipeline.

* Veuillez afficher les traitements disponibles dans la pipeline `fr_core_news_sm` chargée ci-dessus sous le nom de `nlp` .
* Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
* Vérifiez que la désactivation a bien fonctionné en affichant les traitements activés.

In [4]:
# Veuillez écrire votre code ici.
# Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
nlp = spacy.load("fr_core_news_sm")
print("Traitements disponibles :", nlp.pipe_names)
for pipe in nlp.pipe_names:
    if pipe not in {"tok2vec", "morphologizer"}:
        nlp.disable_pipe(pipe)

print("Traitements activés :", nlp.pipe_names)

Traitements disponibles : ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Traitements activés : ['tok2vec', 'morphologizer']


In [ ]:
from spacy.lang.fr.examples import sentences
import tqdm

**1b.** L'objet `sentences` chargé ci-dessus contient une liste de phrases en français. 

* Veuillez afficher les deux premières phrases de `sentences`.
* Veuillez analyser chacune de ces deux phrases avec la pipeline `nlp` puis afficher chaque token et son POS tag.
    * indication : aidez-vous de la [documentation](https://spacy.io/models/fr#fr_core_news_sm) de `fr_core_news_sm`
    * consigne d'affichage : indiquer le tag entre crochets après chaque token, comme ceci : Les \[DET\] robots \[NOUN\] ...
    * note : la documentation détaillée du POS tagging dans spaCy est [disponible ici](https://spacy.io/usage/linguistic-features)
* Veuillez commenter la tokenisation et les POS tags observés : vous semblent-ils corrects pour les deux phrases ?

In [33]:
# Veuillez écrire votre code et votre commentaire ici.
print(f"Premières phrases de l'exemple : {sentences[:2]}")

for sentence in tqdm.tqdm(sentences[:2]):
    doc = nlp(sentence)
    print(f"Analyse de la phrase : {sentence}")
    for token in doc:
        print(f"{token.text} [{token.pos_}] ", end="")
    print("\n")


Premières phrases de l'exemple : ['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars', "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs"]


100%|██████████| 2/2 [00:00<00:00, 419.75it/s]

Analyse de la phrase : Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars
Apple [NOUN] cherche [NOUN] à [ADP] acheter [VERB] une [DET] start [NOUN] - [NOUN] up [ADJ] anglaise [NOUN] pour [ADP] 1 [NUM] milliard [NOUN] de [ADP] dollars [NOUN] 

Analyse de la phrase : Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs
Les [DET] voitures [NOUN] autonomes [ADJ] déplacent [ADV] la [DET] responsabilité [NOUN] de [ADP] l' [DET] assurance [NOUN] vers [ADP] les [DET] constructeurs [NOUN] 



In [34]:
## Commentaire
# On peut voir que les phrases sont bien tokenisées et que les étiquettes de POS sont bien attribuées, toutefois,
# on peut voir que les étiquettes de POS ne sont pas toujours correctes. Par exemple, "cherche" est étiqueté comme un
# nom alors que c'est un verbe. "Start-up" a été segmenté en deux tokens, ce qui me semble pas être une bonne idée.
# "anglaise" est étiqueté comme un nom alors que c'est un adjectif.
# Pour la 2ème phrase, "déplacent" devrait être verbe et non un adverbe.

## 2. Prise en main des données

Les données sont fournies dans un format tabulaire dans l'archive `UD_French-GSD.zip` sur Cyberlearn.  Elles sont basées sur les données fournies par le projet [Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD).  Leur format, appelé CoNLL-U, est [documenté ici](https://universaldependencies.org/format.html).  Veuillez placer les trois fichiers contenus dans l'archive dans un sous-dossier de ce notebook nommé `spacy_data`.

Les trois fichiers contiennent des phrases en français annotées avec les POS tags :
* le fichier `fr-ud-train.conllu` est destiné à l'entraînement
* le fichier `fr-ud-dev.conllu` est destiné aux tests préliminaires et aux réglages des paramètres
* le fichier `fr-ud-test.conllu` est destiné à l'évaluation finale.

**2a.** En inspectant les fichiers avec un éditeur texte, veuillez déterminer dans quelle colonne se trouvent les *tokens* des textes originaux, et dans quelle colonne se trouvent leurs étiquettes morpho-syntaxiques correctes (*POS tags*).  Que contient la troisième colonne ?

In [ ]:
# Veuillez écrire vos réponses dans cette cellule.
## Le numéro de colonne pour les tokens est 2
## Le numéro de colonne pour les POS est 4
## Dans la colonne 3, on a les lemmes des tokens (forme canonique)

**2b.** Veuillez convertir les trois fichiers de données en des fichiers binaires utilisables par spaCy, en utilisant la [commande 'convert' fournie par spaCy](https://spacy.io/api/cli#convert).  La commande est donnée ci-dessous, le premier dossier `./input_data` contient les 3 fichiers `.conllu` et le dossier `./spacy-data` contiendra les 3 résultats.

* Veuillez exécuter la commande de conversion.
* Combien de phrases environ (à 10 phrases près) contient chaque fichier (*train*, *dev*, *test*) ?  Observez la commande et son résultat pour répondre.

In [36]:
!python -m spacy convert ./input_data ./spacy_data --converter conllu  --n-sents 10 --lang fr

ℹ Grouping every 10 sentences into a document.
✔ Generated output file (148 documents): spacy_data/fr-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (42 documents): spacy_data/fr-ud-test.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1456 documents):
spacy_data/fr-ud-train.spacy


In [38]:
# Veuillez indiquer les nombres de phrases ici.

# fr-ud-dev.spacy: 148 documents -> donc environ 1480 phrases (148 × 10).
# fr-ud-test.spacy: 42 documents -> donc environ 420 phrases (42 × 10).
# fr-ud-train.spacy: 1456 documents -> donc environ 14 560 phrases (1456 × 10).

**2c**. Les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Dans notre cas, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la première phrase des données de test dans une variable nommée `premiere_phrase_test`.
* Veuillez afficher cette phrase, ainsi que son type dans spaCy.

In [5]:
from spacy.tokens import DocBin
from spacy.tokens import Doc
test_data = DocBin().from_disk("./spacy_data/fr-ud-test.spacy")
# Exemple d'utilisation (afficher toutes les phrases)
# for doc in test_data.get_docs(nlp.vocab): 
#     for sent in doc.sents:
#         print(sent)

In [6]:
# Veuillez écrire votre code ici.
docs = list(test_data.get_docs(nlp.vocab))
premiere_phrase_test = next(next(test_data.get_docs(nlp.vocab)).sents)
print(f"Première phrase du fichier de test : {premiere_phrase_test}")
print(f"Type de l'objet : {type(premiere_phrase_test)}")

Première phrase du fichier de test : Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.
Type de l'objet : <class 'spacy.tokens.span.Span'>


## 3. Évaluation du POS tagger français de la pipeline `fr_core_news_sm`

**3a.** Veuillez effectuer le *POS tagging* avec spaCy de la `premiere_phrase_test` et afficher les résultats dans le format demandé au (1b).  Indication : convertissez la `premiere_phrase_test` dans un objet de type `Doc` en lui appliquant la méthode `.as_doc()`.  Cet objet peut être ensuite traité par la pipeline `nlp`.

In [54]:
# Veuillez écrire votre code ici.
premiere_phrase_test_doc = premiere_phrase_test.as_doc()
print(f"Type de l'objet : {type(premiere_phrase_test_doc)}")
phrase_traitee = nlp(premiere_phrase_test.text)
print(f"Phrase traitée : {phrase_traitee.text}")
print(f"Analyse de la phrase : {phrase_traitee.text}")
for token in phrase_traitee:
    print(f"{token.text} [{token.pos_}] ", end="")
print("\n")

Type de l'objet : <class 'spacy.tokens.doc.Doc'>
Phrase traitée : Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.
Analyse de la phrase : Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.
Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [PRON] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT] 



**3b.** Veuillez afficher les tags corrects de `premiere_phrase_test`, puis comparez-les visuellement les tags trouvés automatiquement au (3a).  Quelles différences trouvez-vous ?

In [ ]:
# Veuillez écrire votre réponse ici.
# Affichage des POS tags corrects de la phrase test
print("Tags corrects :")
for token in premiere_phrase_test:
    print(f"{token.text} [{token.tag_}]", end=" ")
print("\n")
print("Tags prédits :")
for token in phrase_traitee:
    print(f"{token.text} [{token.pos_}]", end=" ")
print("\n")


Tags corrects :
Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [ADJ] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT] 

Tags prédits :
Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [PRON] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT] 



In [65]:
if len(premiere_phrase_test) != len(phrase_traitee):
    print("Les deux phrases n'ont pas le même nombre de tokens.")
else:
    for i in range(len(premiere_phrase_test)):
        if premiere_phrase_test[i].tag_ != phrase_traitee[i].pos_:
            print(f"Token '{premiere_phrase_test[i]}' : POS correcte -> {premiere_phrase_test[i].tag_} | POS prédite -> {phrase_traitee[i].pos_}")

## On voit que les POS prédites ne sont pas correctes pour le token "fous", qui est étiqueté comme un nom pronom personnel alors que c'est un adjectif.

Token 'fous' : POS correcte -> ADJ | POS prédite -> PRON


In [66]:
from spacy.scorer import Scorer
from spacy.training import Example

In [67]:
scorer = Scorer()

**3c.** Au lieu de compter manuellement combien de tags sont différents entre la référence et le résultat de la pipeline `nlp`, vous allez utiliser la classe `Scorer` de spaCy.  Une instance de cette classe permet de calculer les scores d'une liste d'objets de type `Exemple`, en fonction des annotations disponibles dans les objets.  Un objet de type `Exemple` contient deux objets de type `Doc`, l'un avec les annotations correctes et l'autre avec les annotations produites par une pipeline.  La [documentation de la méthode](https://spacy.io/api/scorer#score) `Scorer.score(..)` vous sera utile. 

* Veuillez calculer la justesse (*accuracy*) du *POS tagging* de `premiere_phrase_test`. 
* Veuillez justifier la valeur du score obtenu en utilisant votre réponse du (3b).

In [ ]:
# Veuillez écrire votre code ici.
example = Example(premiere_phrase_test_doc, phrase_traitee)
scores = scorer.score([example])
## 96.55%
print(f"Accuracy POS sur le dataset test : {scores['pos_acc']:.2%}")


Accuracy POS sur le dataset test : 96.55%


**3d.** Veuillez calculer la précision du *POS tagging* de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se compare le score obtenu avec celui mentionné [dans la documentation](https://spacy.io/models/fr#fr_core_news_sm) du modèle `fr_core_news_sm` ?

In [ ]:
# Veuillez écrire votre code ici, suivi de votre réponse à la question.

score_pos_docs = scorer.score([
    Example(nlp(doc.copy()), doc) for doc in test_data.get_docs(nlp.vocab)
])

## 91.77% de précision
print(f"Précision du POS tagging sur l'ensemble des documents : {score_pos_docs.get('pos_acc'):.2%}")

phrases_test = [phrase for doc in test_data.get_docs(nlp.vocab) for phrase in doc.sents]

score_pos_phrases = scorer.score(
    [Example(nlp(phrase.as_doc().copy()), phrase.as_doc()) for phrase in phrases_test]
    )

## 91.73% de précision
print(f"Précision du POS tagging sur l'ensemble des phrases : {score_pos_phrases.get('pos_acc'):.2%}")


Précision du POS tagging sur l'ensemble des documents : 91.77%
Précision du POS tagging sur l'ensemble des phrases : 91.73%


In [ ]:
# nous avons testé la précision du POS tagging du modèle fr_core_news_sm sur un jeu de données de test. 
# Pour cela, nous avons comparé les annotations de référence avec celles générées par la pipeline spaCy, en utilisant la classe Scorer. 
# Nous avons évalué la précision des tags POS sur deux niveaux : d'abord sur l’ensemble des documents, puis sur chaque phrase individuellement. 
# D’après la documentation de spaCy, le modèle fr_core_news_sm a une précision POS d’environ 96%.
# Cette différence est probablement due aux différences de dataset, aux problèmes de segmentation et aux limites du modèle compact que nous utilisons.

## 4. Entraîner puis évaluer un nouveau POS tagger français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent.  Les [instructions d'entraînement](https://spacy.io/usage/training#quickstart) de spaCy vous montrent comment entraîner une pipeline avec un POS tagger.

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous voulez seulement un POS tagger dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [87]:
#!python -m spacy init config base_config.cfg --lang fr --pipeline tagger
!python -m spacy init fill-config base_config.cfg config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Enfin, veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques, pour estimer le temps nécessaire et observer les messages affichés.  Puis augmentez progressivement le nombre d'époques.  Quel est le critère qui vous permet de décider que vous avez un nombre suffisant d'époques ?  Dans quel dossier se trouve le meilleur modèle ?

In [7]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --training.max_epochs 1 \
  --verbose

[2025-03-16 10:46:00,088] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev', 'training.max_epochs']
ℹ Saving to output directory: myPOStagger1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-16 10:46:03,026] [INFO] Set up nlp object from config
[2025-03-16 10:46:03,036] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-16 10:46:03,038] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-03-16 10:46:03,039] [INFO] Pipeline: ['tok2vec', 'tagger']
[2025-03-16 10:46:03,042] [INFO] Created vocabulary
[2025-03-16 10:46:03,043] [INFO] Finished initializing nlp object
[2025-03-16 10:46:17,001] [INFO] Initialized pipeline components: ['tok2vec', 'tagger']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-16 10:46:17,012] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-16 10:46:17,013] [DEBUG] Loading corpus from 

In [8]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --training.max_epochs 3 \
  --verbose

[2025-03-16 10:53:11,951] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev', 'training.max_epochs']
ℹ Saving to output directory: myPOStagger1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-16 10:53:14,938] [INFO] Set up nlp object from config
[2025-03-16 10:53:14,948] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-16 10:53:14,951] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-03-16 10:53:14,951] [INFO] Pipeline: ['tok2vec', 'tagger']
[2025-03-16 10:53:14,954] [INFO] Created vocabulary
[2025-03-16 10:53:14,954] [INFO] Finished initializing nlp object
[2025-03-16 10:53:29,308] [INFO] Initialized pipeline components: ['tok2vec', 'tagger']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-16 10:53:29,318] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-16 10:53:29,319] [DEBUG] Loading corpus from 

In [ ]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --training.max_epochs 4 \
  --verbose

[2025-03-16 10:58:35,083] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev', 'training.max_epochs']
ℹ Saving to output directory: myPOStagger1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-16 10:58:40,402] [INFO] Set up nlp object from config
[2025-03-16 10:58:40,417] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-16 10:58:40,420] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-03-16 10:58:40,420] [INFO] Pipeline: ['tok2vec', 'tagger']
[2025-03-16 10:58:40,425] [INFO] Created vocabulary
[2025-03-16 10:58:40,425] [INFO] Finished initializing nlp object
[2025-03-16 10:59:03,770] [INFO] Initialized pipeline components: ['tok2vec', 'tagger']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-16 10:59:03,787] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-16 10:59:03,790] [DEBUG] Loading corpus from 

In [ ]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --training.max_epochs 5 \
  --verbose

In [ ]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --verbose


[2025-03-15 20:55:02,027] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: myPOStagger1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-15 20:55:06,725] [INFO] Set up nlp object from config
[2025-03-15 20:55:06,745] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-15 20:55:06,748] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-03-15 20:55:06,748] [INFO] Pipeline: ['tok2vec', 'tagger']
[2025-03-15 20:55:06,753] [INFO] Created vocabulary
[2025-03-15 20:55:06,753] [INFO] Finished initializing nlp object
[2025-03-15 20:55:29,590] [INFO] Initialized pipeline components: ['tok2vec', 'tagger']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-03-15 20:55:29,608] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-03-15 20:55:29,610] [DEBUG] Loading corpus from path: spacy_data/fr-ud-

In [ ]:
# Veuillez indiquer ici le nombre d'époques final et la réponse à la question.
# Le meilleur modèle se trouve dans myPOStagger1/model-best
# Le nombre d'époques final est 4.
# Pour savoir si suffisamment d'époques ont été effectuées est l'absence d'amélioration ou la stagnation de la précision.

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher la *POS tagging accuracy* sur le corpus de test.  Le composant de la pipeline étant un *POS tagger*, vous devrez évaluer la propriété *tag_acc*. 

In [ ]:
# Veuillez écrire votre code ici.
nlp_best = spacy.load("./myPOStagger1/model-best")
score_pos_docs_best = scorer.score([
    Example(nlp_best(doc.text), doc) for doc in test_data.get_docs(nlp.vocab)
])

print(f"Précision du POS tagging sur l'ensemble des documents : {score_pos_docs_best.get('tag_acc'):.2%}")


Précision du POS tagging sur l'ensemble des documents : 93.36%


In [95]:
# On a une précision de 93.36% sur l'ensemble des documents.

## 5. Entraîner puis évaluer un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger appelé *Averaged Perceptron* fourni par NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus, importées cette fois-ci avec NLTK.  Pour une introduction au POS tagging avec NLTK, voir le [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html).

Remarques :
* pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK ;
* pour appliquer un tagger existant, on écrit `nltk.pos_tag(sentence)` où `sentence` est une liste de tokens et on obtient des paires (token, TAG) ;
* l'implémentation de *Averaged Perceptron* a été faite par [Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python), la société qui a créé spaCy.

**5a.** Veuillez charger les données d'entraînement et celles de test grâce à la classe `ConllCorpusReader` de NLTK.  [La documentation de cette classe](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader) vous montrera comment indiquer les colonnes qui contiennent les tokens ('words') et les tags corrects ('pos').  Une fois les données chargées dans une variable, vous pouvez accéder aux phrases et aux tags avec la méthode `.tagged_sents()`.

In [108]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [109]:
# Veuillez écrire votre code ici.
train_reader = ConllCorpusReader("./input_data", "fr-ud-train.conllu", ("ignore", "words", "ignore", "pos"), separator="\t")
test_reader = ConllCorpusReader("./input_data", "fr-ud-test.conllu", ("ignore", "words", "ignore", "pos"), separator="\t")
train_sents = train_reader.tagged_sents()
test_sents = test_reader.tagged_sents()

print(f"Nombre de phrases dans le jeu de données d'entraînement : {len(train_sents)}")
print(f"Nombre de phrases dans le jeu de données de test : {len(test_sents)}")

# Veuillez écrire votre code ici.

print(f"Première phrase du jeu de données d'entraînement : {train_sents[0]}")


Nombre de phrases dans le jeu de données d'entraînement : 14554
Nombre de phrases dans le jeu de données de test : 416
Première phrase du jeu de données d'entraînement : [('Les', 'DET'), ('commotions', 'NOUN'), ('cérébrales', 'ADJ'), ('sont', 'AUX'), ('devenu', 'VERB'), ('si', 'ADV'), ('courantes', 'ADJ'), ('dans', 'ADP'), ('ce', 'DET'), ('sport', 'NOUN'), ("qu'", 'SCONJ'), ('on', 'PRON'), ('les', 'PRON'), ('considére', 'VERB'), ('presque', 'ADV'), ('comme', 'ADP'), ('la', 'DET'), ('routine', 'NOUN'), ('.', 'PUNCT')]


**5b.** Pour entraîner un POS tagger du type Averaged Perceptron, vous utiliserez le sous-module `nltk.tag.perceptron` du [module NLTK contenant les taggers](http://www.nltk.org/api/nltk.tag.html).  Les fonctions d'entraînement et de test sont documentées dans ce module.  Après l'entraînement, le réseau de neurones est enregistré dans un fichier `.pickle`, qui est écrasé à chaque entraînement si vous n'en faites pas une copie.  On peut également lire un fichier `.pickle` dans un tagger.

Veuillez écrire le code pour entraîner le POS tagger sur les données d'entraînement.  Comme au (4), pensez augmenter graduellement le nombre d'époques (appelées 'itérations' dans NLTK).

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier enregistré ?

In [128]:
import os
import nltk
#nltk.download('averaged_perceptron_tagger') # à exécuter la première fois
from perceptron_patched import PerceptronTagger

In [129]:
ptagger = PerceptronTagger(load=False)

In [ ]:
import time 
import os

os.makedirs("./ptaggerOutput", exist_ok=True)

nbIteration = [2,3,5,10,20]

for it in nbIteration:
    start_time = time.time()
    model_path = f"./ptaggerOutput/ptaggermodel-{it}.pickle"
    
    ptagger.train(train_sents, save_loc=model_path, nr_iter=it) 

    training_time = time.time() - start_time
    print(f"Temps d'entraînement pour {it} epochs: {training_time:.2f} secondes")
    
    file_size_bytes = os.path.getsize(model_path + "_averaged_perceptron_tagger.xxx.classes.json")
    file_size_bytes += os.path.getsize(model_path + "_averaged_perceptron_tagger.xxx.tagdict.json")
    file_size_bytes += os.path.getsize(model_path + "_averaged_perceptron_tagger.xxx.weights.json")
    file_size_mb = file_size_bytes / (1024 * 1024) 
    print(f"Taille des fichiers pour {it} epochs: {file_size_mb:.2f} MB")

# Temps d'entraînement pour 2 epochs: 9.67 secondes
# Taille des fichiers pour 2 epochs: 4.48 MB
# Temps d'entraînement pour 3 epochs: 12.41 secondes
# Taille des fichiers pour 3 epochs: 5.69 MB
# Temps d'entraînement pour 5 epochs: 19.71 secondes
# Taille des fichiers pour 5 epochs: 6.52 MB
# Temps d'entraînement pour 10 epochs: 38.37 secondes
# Taille des fichiers pour 10 epochs: 7.16 MB
# Temps d'entraînement pour 20 epochs: 80.25 secondes
# Taille des fichiers pour 20 epochs: 7.60 MB

**5c.** Veuillez évaluer le tagger sur les données de test et afficher le taux de correction.

In [ ]:
# Veuillez écrire votre code ici.
for it in nbIteration:
    model_path = f"./ptaggerOutput/ptaggermodel-{it}.pickle"
    
    # Load the model
    tagger = PerceptronTagger(load=False)
    tagger.load_from_json(model_path)
    
    # Calculate accuracy on test data
    accuracy = tagger.accuracy(test_sents)
    
    print(f"Modèle pour {it} epochs:")
    print(f"Precision du POS sur les données de test : {accuracy:.2%}")

# Modèle pour 2 epochs:
# Precision du POS sur les données de test : 95.67%
# Modèle pour 3 epochs:
# Precision du POS sur les données de test : 95.75%
# Modèle pour 5 epochs:
# Precision du POS sur les données de test : 95.92%
# Modèle pour 10 epochs:
# Precision du POS sur les données de test : 95.95%
# Modèle pour 20 epochs:
# Precision du POS sur les données de test : 96.06%

Précision du POS tagging sur le jeu de données de test : 95.90%


## 6. Conclusion

Veuillez remplir le tableau suivant avec les scores obtenus et discuter brièvement comment se comparent les trois POS taggers sur ces données de test.

| spaCy (partie 3) | spaCy (partie 4) | NLTK (partie 5) | 
|------------------|------------------|-----------------|
| tagger fourni    | tagger entraîné  | tagger entraîné |
| 91.77%             | 93.36%              | 95.90%             |


In [ ]:
# Veuillez écrire votre réponse ici.
# NLTK est plus rapide à entraîner que spaCy et plus précis également.
# Etant donné le tagger de spaCy (partie 3) est plus généraliste, on peut supposer qu'il est normal qu'il soit moins précis que nos taggers
# entrainés chez nous. Cependant, il est plus rapide utiliser.

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.